# 池化层详解：最大池化与平均池化

池化层是CNN的核心组件之一，主要作用：
1. **下采样** - 降低特征图空间维度，减少计算量和参数
2. **平移不变性** - 使网络对输入的小幅位移更加鲁棒
3. **扩大感受野** - 后续层能够"看到"更大范围的输入区域

本教程涵盖：
- 最大池化与平均池化的原理与对比
- Keras实现与参数详解
- 池化对特征图的影响可视化

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

## 第一部分：池化层原理

### 1.1 最大池化 (Max Pooling)

最大池化在每个池化窗口中选取最大值，保留最显著的特征响应。

**适用场景**：
- 需要检测特征是否存在（边缘、纹理等）
- 大多数CNN分类任务的默认选择

### 1.2 平均池化 (Average Pooling)

平均池化计算窗口内所有值的平均，保留区域的整体响应强度。

**适用场景**：
- 需要保留背景信息
- 全局平均池化用于替代全连接层

In [ ]:
# 创建示例特征图，模拟卷积层输出
# 形状: (batch_size, height, width, channels)
sample_feature_map = np.array([
    [[1, 2, 3, 4],
     [5, 6, 7, 8],
     [9, 10, 11, 12],
     [13, 14, 15, 16]]
], dtype=np.float32).reshape(1, 4, 4, 1)

print("输入特征图 (4x4):")
print(sample_feature_map.squeeze())
print(f"\n形状: {sample_feature_map.shape}")

In [ ]:
# 创建池化层
# pool_size=2 表示 2x2 窗口
# strides 默认等于 pool_size，即窗口不重叠

max_pool = keras.layers.MaxPooling2D(
    pool_size=2,      # 2x2 池化窗口
    strides=None,     # None表示等于pool_size
    padding='valid'   # 不填充
)

avg_pool = keras.layers.AveragePooling2D(
    pool_size=2,
    strides=None,
    padding='valid'
)

# 应用池化
max_pooled = max_pool(sample_feature_map)
avg_pooled = avg_pool(sample_feature_map)

print("最大池化结果 (2x2):")
print(max_pooled.numpy().squeeze())
print(f"\n最大池化计算过程:")
print(f"  左上: max(1,2,5,6) = 6")
print(f"  右上: max(3,4,7,8) = 8")
print(f"  左下: max(9,10,13,14) = 14")
print(f"  右下: max(11,12,15,16) = 16")

print("\n平均池化结果 (2x2):")
print(avg_pooled.numpy().squeeze())
print(f"\n平均池化计算过程:")
print(f"  左上: mean(1,2,5,6) = 3.5")
print(f"  右上: mean(3,4,7,8) = 5.5")

## 第二部分：池化层参数详解

### 2.1 pool_size 与 strides 的关系

- `pool_size`: 池化窗口大小
- `strides`: 窗口移动步长
- 当 `strides = pool_size` 时，窗口不重叠（最常用）
- 当 `strides < pool_size` 时，窗口重叠

In [ ]:
# 演示不同strides的效果
# 创建较大的特征图
large_feature = np.arange(1, 65, dtype=np.float32).reshape(1, 8, 8, 1)

print("输入特征图 (8x8):")
print(large_feature.squeeze())

# 不重叠池化 (strides=2)
pool_no_overlap = keras.layers.MaxPooling2D(pool_size=2, strides=2)
# 重叠池化 (strides=1)
pool_overlap = keras.layers.MaxPooling2D(pool_size=2, strides=1)

result_no_overlap = pool_no_overlap(large_feature)
result_overlap = pool_overlap(large_feature)

print(f"\n不重叠池化 (strides=2) 输出形状: {result_no_overlap.shape}")
print(f"重叠池化 (strides=1) 输出形状: {result_overlap.shape}")

In [ ]:
# padding参数的影响
# 创建奇数尺寸的特征图
odd_feature = np.arange(1, 26, dtype=np.float32).reshape(1, 5, 5, 1)

print("输入特征图 (5x5):")
print(odd_feature.squeeze())

# valid padding: 不填充，可能丢失边缘
pool_valid = keras.layers.MaxPooling2D(pool_size=2, padding='valid')
# same padding: 填充以保持输出尺寸为 ceil(input/stride)
pool_same = keras.layers.MaxPooling2D(pool_size=2, padding='same')

result_valid = pool_valid(odd_feature)
result_same = pool_same(odd_feature)

print(f"\npadding='valid' 输出形状: {result_valid.shape}")
print(result_valid.numpy().squeeze())

print(f"\npadding='same' 输出形状: {result_same.shape}")
print(result_same.numpy().squeeze())

## 第三部分：真实图像上的池化效果

观察池化对图像的降采样效果

In [ ]:
# 加载示例图像
from sklearn.datasets import load_sample_image

# 加载并预处理图像
china = load_sample_image("china.jpg") / 255.0
china_gray = np.mean(china, axis=2, keepdims=True).astype(np.float32)
china_batch = np.expand_dims(china_gray, axis=0)

print(f"原始图像形状: {china_batch.shape}")

In [ ]:
# 应用不同池化策略
max_pool_2x2 = keras.layers.MaxPooling2D(pool_size=2)
max_pool_4x4 = keras.layers.MaxPooling2D(pool_size=4)
avg_pool_2x2 = keras.layers.AveragePooling2D(pool_size=2)
avg_pool_4x4 = keras.layers.AveragePooling2D(pool_size=4)

results = {
    '原图': china_gray,
    'MaxPool 2x2': max_pool_2x2(china_batch).numpy().squeeze(),
    'MaxPool 4x4': max_pool_4x4(china_batch).numpy().squeeze(),
    'AvgPool 2x2': avg_pool_2x2(china_batch).numpy().squeeze(),
    'AvgPool 4x4': avg_pool_4x4(china_batch).numpy().squeeze(),
}

In [ ]:
# 可视化池化效果
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, (title, img) in zip(axes, results.items()):
    if img.ndim == 3:
        img = img.squeeze()
    ax.imshow(img, cmap='gray')
    ax.set_title(f'{title}\n形状: {img.shape}')
    ax.axis('off')

# 隐藏多余的子图
axes[-1].axis('off')

plt.tight_layout()
plt.show()

## 第四部分：全局池化层

### 4.1 GlobalAveragePooling2D

全局平均池化将每个特征图压缩为单个值，常用于：
- 替代Flatten + Dense层，大幅减少参数
- 现代CNN架构（ResNet, Inception等）的标准做法

In [ ]:
# 模拟CNN最后一层卷积输出
# 假设是 7x7 空间维度，512个通道
simulated_conv_output = np.random.randn(1, 7, 7, 512).astype(np.float32)

print(f"卷积层输出形状: {simulated_conv_output.shape}")
print(f"总元素数: {np.prod(simulated_conv_output.shape[1:])}")

# 全局平均池化
global_avg_pool = keras.layers.GlobalAveragePooling2D()
gap_output = global_avg_pool(simulated_conv_output)

print(f"\n全局平均池化后形状: {gap_output.shape}")
print(f"输出元素数: {gap_output.shape[-1]}")

# 参数量对比
flatten_params = 7 * 7 * 512 * 1000  # 如果接1000类分类器
gap_params = 512 * 1000

print(f"\n参数量对比 (假设1000类分类):")
print(f"  Flatten + Dense: {flatten_params:,} 参数")
print(f"  GAP + Dense: {gap_params:,} 参数")
print(f"  参数减少: {(1 - gap_params/flatten_params)*100:.1f}%")

In [ ]:
# GlobalMaxPooling2D 对比
global_max_pool = keras.layers.GlobalMaxPooling2D()
gmp_output = global_max_pool(simulated_conv_output)

print("全局池化对比:")
print(f"  GlobalAveragePooling2D 输出形状: {gap_output.shape}")
print(f"  GlobalMaxPooling2D 输出形状: {gmp_output.shape}")

## 第五部分：在CNN中使用池化层

In [ ]:
def build_cnn_with_pooling(use_global_avg_pool=True):
    """
    构建带池化层的CNN，演示池化层的实际应用
    
    Parameters
    ----------
    use_global_avg_pool : bool
        是否使用全局平均池化替代Flatten
    
    Returns
    -------
    keras.Model
    """
    model = keras.Sequential([
        # 输入层
        keras.layers.InputLayer(input_shape=(28, 28, 1)),
        
        # 第一卷积块
        keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=2),  # 28x28 -> 14x14
        
        # 第二卷积块
        keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        keras.layers.MaxPooling2D(pool_size=2),  # 14x14 -> 7x7
        
        # 第三卷积块
        keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    ])
    
    if use_global_avg_pool:
        model.add(keras.layers.GlobalAveragePooling2D())  # 7x7x128 -> 128
    else:
        model.add(keras.layers.Flatten())  # 7x7x128 -> 6272
        model.add(keras.layers.Dense(128, activation='relu'))
    
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    return model

# 对比两种架构
model_gap = build_cnn_with_pooling(use_global_avg_pool=True)
model_flatten = build_cnn_with_pooling(use_global_avg_pool=False)

print("使用GlobalAveragePooling2D的模型:")
print(f"总参数量: {model_gap.count_params():,}")

print("\n使用Flatten的模型:")
print(f"总参数量: {model_flatten.count_params():,}")

In [ ]:
# 验证模型可以正常运行
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train = X_train[:1000].reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = y_train[:1000]

model_gap.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 快速训练验证
print("验证模型训练...")
history = model_gap.fit(
    X_train, y_train,
    epochs=2,
    batch_size=32,
    verbose=1
)
print("模型训练验证完成")

## 总结

### 池化层选择指南

| 池化类型 | 特点 | 适用场景 |
|---------|------|----------|
| MaxPooling2D | 保留最强响应 | 特征检测、分类任务 |
| AveragePooling2D | 保留平均响应 | 需要平滑输出 |
| GlobalAveragePooling2D | 替代Flatten | 现代CNN架构 |
| GlobalMaxPooling2D | 全局最大响应 | 特定检测任务 |

### 关键参数

- `pool_size=2`: 最常用，使空间维度减半
- `strides=None`: 默认等于pool_size，窗口不重叠
- `padding='valid'`: 默认不填充